In [1]:
from tabulate import tabulate
from reservoirpy.nodes import (Reservoir, IPReservoir, NVAR, LMS, RLS, Ridge, ESN, Input)
from reservoirpy.datasets import (lorenz, mackey_glass, narma)
from reservoirpy.observables import (rmse, rsquare, nrmse, mse)
import numpy as np
import copy
import math
import matplotlib.pyplot as plt
from numpy import linalg as LA

Following the same setup as "Growing Echo-State Network With Multiple Subreservoirs"

In [2]:
def getData():
    data = np.expand_dims(np.array(lorenz(n_timesteps=8000))[:, 0], 1)
    train = data[:5600]
    test = data[5600:]
    trainX = train[:-1]
    trainY = train[1:]
    testX = test[:-1]
    testY = test[1:]
    return trainX, trainY, testX, testY

In [4]:
def testDefault():
    esn = Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)
    preds = esn.run(testX)
    return nrmse(testY, preds)

def testDeep():
    reservoir1 = Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)
    reservoir2 = Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)
    readout = Ridge(output_dim=1, ridge=1e-5)
    esn = reservoir1>>reservoir2>>readout & reservoir1>>readout
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)
    preds = esn.run(testX)
    return nrmse(testY, preds)

def testDeepCascade():
    esn = (Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)

    esn2 = (Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(trainX)
    esn2.fit(esnPreds, trainY, warmup=1000)

    preds1 = esn.run(testX)
    preds2 = esn2.run(preds1)
    return nrmse(testY, preds2)

def testCascade():
    esn = Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)

    esn2 = Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(trainX)
    esn2.fit(esnPreds, trainY, warmup=1000)

    preds1 = esn.run(testX)
    preds2 = esn2.run(preds1)
    return nrmse(testY, preds2)

def testErrorCascade():
    esn = Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)

    esn2 = Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(trainX)
    errors = esnPreds - trainY
    esn2.fit(esnPreds, errors, warmup=1000)

    preds1 = esn.run(testX)
    predictedErrors = esn2.run(preds1)
    preds = preds1 - predictedErrors
    return nrmse(testY, preds)

def testDeepErrorCascade():
    esn = (Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)

    esn2 = (Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(trainX)
    errors = esnPreds - trainY
    esn2.fit(esnPreds, errors, warmup=1000)

    preds1 = esn.run(testX)
    predictedErrors = esn2.run(preds1)
    preds = preds1 - predictedErrors
    return nrmse(testY, preds)

def testNVAR():
    esn = NVAR(delay=8, order=3, strides=4) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)
    preds = esn.run(testX)
    return nrmse(testY, preds)



def testDeepCascadeNVAR():
    esn = (NVAR(delay=8, order=3, strides=4) >> Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)

    esn2 = (NVAR(delay=8, order=3, strides=4) >> Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(trainX)
    esn2.fit(esnPreds, trainY, warmup=1000)

    preds1 = esn.run(testX)
    preds2 = esn2.run(preds1)
    return nrmse(testY, preds2)

error1 = testDefault()
error2 = testCascade()
error4 = testDeep()
error5 = testDeepCascade()
error6 = testDeepCascadeNVAR()
error7 = testErrorCascade()
error8 = testDeepErrorCascade()
error9 = testNVAR()
print("==================Lorenz NRMSE==================")
print('Normal training error:', error1)
print('NVAR training error:', error9)
print('Cascading ESN error:', error2)
print('Deep ESN error:', error4)
print('Deep + cascade error:', error5)
print('Deep + cascade + NVAR error:', error6)
print('Error cascade error:', error7)
print('Deep error cascade error:', error8)

Running Model-2: 5599it [00:00, 20415.75it/s]?, ?it/s]
Running Model-2: 100%|██████████| 1/1 [00:00<00:00,  3.57it/s]


Fitting node Ridge-2...


Running Model-2: 2399it [00:00, 20640.53it/s]         
Running Model-3: 5599it [00:00, 20186.50it/s]?, ?it/s]
Running Model-3: 100%|██████████| 1/1 [00:00<00:00,  3.55it/s]


Fitting node Ridge-3...


Running Model-3: 5599it [00:00, 21915.73it/s]         
Running Model-4: 5599it [00:00, 20413.88it/s]?, ?it/s]
Running Model-4: 100%|██████████| 1/1 [00:00<00:00,  3.59it/s]


...

Fitting node Ridge-13...


Running Model-20: 2399it [00:00, 12249.21it/s]         
Running Model-22: 2399it [00:00, 12796.24it/s]         
Running Model-23: 5599it [00:00, 26057.68it/s]?, ?it/s]
Running Model-23: 100%|██████████| 1/1 [00:00<00:00,  4.56it/s]


Fitting node Ridge-14...


Running Model-23: 2399it [00:00, 26690.08it/s]         

==================Lorenz NRMSE==================
Normal training error: 0.10552245269443629
NVAR training error: 0.000155199556840157
Cascading ESN error: 0.10404548288972969
Deep ESN error: 0.0724038265960128
Deep + cascade error: 0.09863333966967663
Deep + cascade + NVAR error: 0.045699767354320416
Error cascade error: 0.0964124374546007
Deep error cascade error: 0.0856291986784707


In [31]:
from bayes_opt import BayesianOptimization
import random

def testDefault(**params):
    index = str(random.randint(1000, 10000))
    timeDelayedInput = Input(name='delayed_input_'+index)
    normalInput = Input(name='normal_input_'+index)
    reservoir1 = Reservoir(units=int(params['units']), sr=params['sr'])
    reservoir2 = Reservoir(units=int(params['units']), sr=params['sr'])
    readout = Ridge(output_dim=1, ridge=params['ridge'])
    esn = normalInput >> reservoir1 >> readout & timeDelayedInput >> reservoir2 >> readout
    trainX, trainY, testX, testY = getData()

    delay = 100
    trainXNormal = trainX[delay:]
    trainXDelayed = trainX[:-delay]
    trainY = trainY[delay:]
    
    testXNormal = testX[delay:]
    testXDelayed = testX[:-delay]
    testY = testY[delay:]

    esn.fit({'delayed_input_'+index: trainXDelayed, 'normal_input_'+index: trainXNormal}, trainY, warmup=1000)
    preds = esn.run({'delayed_input_'+index: testXDelayed, 'normal_input_'+index: testXNormal})
    return 1/nrmse(testY, preds)

pbounds = {'units': (100, 1000), 'sr': (0.1, 1.2), 'ridge': (1e-6, 1e-4)}

optimizer = BayesianOptimization(
    f=testDefault,
    pbounds=pbounds,
    random_state=1,
)

optimizer.maximize(
    init_points=20,
    n_iter=25,
)

|   iter    |  target   |   ridge   |    sr     |   units   |
-------------------------------------------------------------


Running Model-688: 5499it [00:00, 8886.85it/s]<?, ?it/s]
Running Model-688: 100%|██████████| 1/1 [00:00<00:00,  1.60it/s]


Fitting node Ridge-154...


Running Model-688: 2299it [00:00, 10824.82it/s]         


| 1         | 0.03581   | 4.229e-05 | 0.8924    | 100.1     |


...

| 44        | 5.61      | 7.786e-05 | 0.9986    | 600.7     |


Running Model-908: 5499it [00:01, 4460.91it/s]<?, ?it/s]
Running Model-908: 100%|██████████| 1/1 [00:01<00:00,  1.31s/it]


Fitting node Ridge-198...


Running Model-908: 2299it [00:00, 4877.53it/s]          

| 45        | 0.006012  | 4.35e-05  | 0.3698    | 964.4     |


In [32]:
1/optimizer.max['target']

0.17610533061282987